In [ ]:
# required libraries & dataset

#!curl -L "https://www.dropbox.com/s/ypvbhkgt4sb40sy/dataset_b.zip?dl=0" > dataset_b.zip; unzip dataset_b.zip; rm dataset_b.zip
#!pip install livelossplot

In [ ]:
# Importing dependencies

import tensorflow as tf
import keras.metrics
import os, os.path
import numpy as np
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import metrics

In [ ]:
# Image size and Input shape 
IMG_SIZE = 224 #EfficientNet-B0 size
width = 224
height = 224
input_shape = (height, width, 3)
seed = 1432
class_mode = 'binary'

In [ ]:
#Setting TPU or CPU/GPU strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    print("Device:", tpu.master())
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError:
    print("Using CPU/GPU strategy")
    strategy = tf.distribute.MirroredStrategy()

In [ ]:
# Setting dataset directory. Change directory whenever necessary
train_dir ='train' 
valid_dir = 'valid'
test_dir = 'test' 
batch_size = 16

In [ ]:
#Image augmentation

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

img_augmentation = Sequential(
    [
        layers.RandomRotation(factor=0.15),
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        layers.RandomFlip(),
        layers.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)

In [ ]:
#Additional data augmentation

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data = ImageDataGenerator(rescale=1./255)

test_data = ImageDataGenerator(rescale=1./255)

#One-hot encoding data

train_generator = train_data.flow_from_directory(
        train_dir,
        target_size=(height, width),
        seed=seed,
        batch_size=batch_size,
        class_mode=class_mode)

validation_generator = test_data.flow_from_directory(
        valid_dir,
        target_size=(height, width),
        seed=seed,
        batch_size=batch_size,
        class_mode=class_mode)

classnames = train_generator.class_indices

print(classnames)

In [ ]:
#EfficientNet-B0 Model

from tensorflow.keras.applications import EfficientNetB0
enmodel = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
enmodel.input

In [ ]:
#Setting epochs and dropout rate
epochs = 100
dropout_rate = 0.2

In [ ]:
# Adding dense layers, needs rework
model = models.Sequential()
model.add(enmodel)
model.add(layers.GlobalMaxPooling2D(name="gap"))
model.add(layers.Flatten(name="flatten"))
if dropout_rate > 0:
    model.add(layers.Dropout(dropout_rate, name="dropout_out"))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
enmodel.trainable = False
model.summary()
print('Number of trainable units:', len(model.trainable_weights))

In [ ]:
# Compiling the model
with strategy.scope():
    input = layers.Input(shape=(input_shape))
    x = img_augmentation(input)
    output = EfficientNetB0(include_top=True, weights=None, classes=1)(x)

    model = tf.keras.Model(input, output)
    model.compile(
          optimizer="adam", loss="binary_crossentropy", metrics=['accuracy',
             tf.keras.metrics.Precision(),
             tf.keras.metrics.Recall()])
model.summary()

In [ ]:
#Training

from livelossplot import PlotLossesKeras
from livelossplot.keras import PlotLossesCallback

plot_loss_1 = PlotLossesCallback()

from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("efficientnet_rev230130.h5",
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=False,
                             save_weights_only=False,
                             mode='auto',
                             save_freq=1)

early = EarlyStopping(monitor='val_loss',
                      min_delta=0,
                      patience=10,
                      verbose=1, 
                      restore_best_weights=True,
                      mode='auto')

In [ ]:
%%time

enmodel_hist=model.fit(train_generator,
                        epochs=epochs,
                        validation_data=validation_generator, 
                        callbacks= [checkpoint, early, plot_loss_1],
                        verbose=1)

In [ ]:
"""
# Download model if run from colab, else remove

from google.colab import files

files.download('/content/efficientnet_rev230130.h5')
"""